In [32]:
from path_example_vars import GCS_PATH

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

import pandas as pd
import numpy as np
import geopandas as gpd
from siuba import *
import json

import shared_utils
import warnings
from path_example_vars import GCS_PATH

from shapely.ops import split, substring, LineString
from calitp_data_analysis import geography_utils

CONVEYAL_GCS_PATH = 'gs://calitp-analytics-data/data-analyses/conveyal_update/'

In [6]:
import conveyal_path_utils

In [14]:
df = conveyal_path_utils.read_conveyal_path_df(f'{GCS_PATH}la_PATHS.csv')

In [11]:
def read_conveyal_path_df(path):

    array_cols = ['routes', 'boardStops', 'alightStops',
           'rideTimes', 'waitTimes', 'feedIds']

    def unpack_conveyal_path_df(df, array_cols = array_cols):

        for col in array_cols:
            df.loc[:,col] = df[col].map(lambda x: x.split('|'))
        return df

    df = pd.read_csv(path)
    df['total_iterations'] = df.query('origin == 0 & destination == 0').nIterations.iloc[0]
    df = df.query('origin == 0 & destination == 1').drop(columns=['group'])
    df.reset_index(drop=True, inplace=True)
    df.index.rename('trip_group_id', inplace=True)
    #  won't be in Conveyal order but maps will be less confusing
    #  can hash other cols if need to match Conveyal trip-level unique id
    df.reset_index(drop=False, inplace=True)
    df = unpack_conveyal_path_df(df)
    return df

In [13]:
df2 = read_conveyal_path_df(f'{GCS_PATH}la_PATHS.csv')

In [17]:
df.equals(df2)

True

In [19]:
df2.head(3)

,trip_group_id,origin,destination,routes,boardStops,alightStops,feedIds,rideTimes,accessTime,egressTime,transferTime,waitTimes,totalTime,nIterations,total_iterations
0,0,0,1,"[209-13168, 804, 805, 90-13168, 96-13168]","[141078, 80128, 80211, 12102, 3611]","[2489, 80122, 80213, 3611, 1574]","[654404a5dad1a760f30e340d, 654404b9dad1a760f30...","[14.0, 18.0, 2.0, 3.0, 46.0]",8.6,7.2,2.3,"[1.4, 4.3, 1.5, 1.9, 2.0]",112.2,2,240
1,1,0,1,"[108-13168, 910-13168, 802, 94-13168]","[7962, 2322, 80211, 30001]","[1539, 65300042, 80201, 11812]","[654404a5dad1a760f30e340d, 654404a5dad1a760f30...","[15.0, 12.0, 26.0, 20.0]",1.4,3.8,9.8,"[1.6, 5.3, 2.4, 5.4]",102.8,56,240
2,2,0,1,"[108-13168, 910-13168, 802, 3163, 164-13168]","[7963, 2322, 80211, 3068903, 16428]","[1539, 65300042, 80201, 3068909, 11812]","[654404a5dad1a760f30e340d, 654404a5dad1a760f30...","[16.0, 12.0, 26.0, 12.0, 8.0]",1.3,3.8,10.5,"[1.7, 5.3, 4.4, 4.6, 2.1]",107.8,18,240


In [20]:
df2 = conveyal_path_utils.add_warehouse_identifiers(df2)

In [23]:
points = pd.read_csv(f'{GCS_PATH}la_points.csv')
points = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy(points.lon, points.lat)
                          , crs=geography_utils.WGS84).to_crs(geography_utils.CA_NAD83Albers_m)
points.geometry = points.buffer(100)

In [26]:
#  get warehouse data
warehouse_data = conveyal_path_utils.get_warehouse_data(df2)
# spatial_routes = conveyal_path_utils.compile_all_spatial_routes(df2, warehouse_data, verbose=False)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did 

In [30]:
for _ix, row in df2.iterrows():
    test_ssr = conveyal_path_utils.shape_segments_from_row(row, warehouse_data, True)
    break    

In [31]:
test_ssr

,feed_key,trip_id,stop_id,stop_sequence,stop_geom,name,shape_id,route_short_name,route_long_name,route_id,segment_geom,stop_pair,trip_group_id,optimal_pct,total_time,xfer_count,route_name
0,edb0fd72b98a7f739407ceca3fae9034,10209000510529-JUNE23,141078,32,POINT (155462.061 -445965.161),LA Metro Bus Schedule,2090051_JUNE23,209,Metro Local Line,209-13168,"LINESTRING (155451.791 -445961.441, 155451.900...","(141078, 2489)",0,0.008333,112.2,4,209
0,beaaee89e671fea3d48124dc23335bed,59052614,80122,11,POINT (160750.294 -439298.745),LA Metro Rail Schedule,804WB_RC_221121,None,Metro E-Line,804,"LINESTRING (160750.580 -439298.986, 160748.045...","(80128, 80122)",0,0.008333,112.2,4,Metro E-Line
0,beaaee89e671fea3d48124dc23335bed,58926335,80211,11,POINT (160763.169 -439295.846),LA Metro Rail Schedule,802EB_190513,None,Metro B-Line,802,"LINESTRING (160759.382 -439300.931, 160839.511...","(80211, 80213)",0,0.008333,112.2,4,Metro B-Line
0,edb0fd72b98a7f739407ceca3fae9034,10090003590454-JUNE23,12102,3,POINT (161917.438 -438595.678),LA Metro Bus Schedule,900359_JUNE23,90,Metro Local Line,90-13168,"LINESTRING (161907.791 -438588.470, 161976.674...","(12102, 3611)",0,0.008333,112.2,4,90
0,edb0fd72b98a7f739407ceca3fae9034,10094002580436-JUNE23,3611,16,POINT (162503.972 -437721.182),LA Metro Bus Schedule,940258_JUNE23,94,Metro Local Line,94-13168,"LINESTRING (162493.765 -437717.075, 162501.659...","(3611, 1574)",0,0.008333,112.2,4,94


In [33]:
def shape_segments_from_row(row, warehouse_data, verbose):

    stop_pairs = list(zip(row.boardStops, row.alightStops))
    
    row_shape_segments = []
    for stop_pair in stop_pairs:
        # print(stop_pair)
        first_filter = warehouse_data['st'].query('stop_id.isin(@stop_pair)')
        # display(first_filter)
        return first_filter
        good_trips = first_filter >> count(_.trip_id) >> filter(_.n > 1)
        #  TODO count frequency using departure sec?
        better_trips = good_trips >> filter(_.n == _.n.max())
        # display(good_trips)
        assert better_trips.shape[0] > 0
        trip_with_pair = first_filter >> filter(_.trip_id == better_trips.trip_id.iloc[0]) >> arrange(_.stop_sequence)
        trip_with_pair = trip_with_pair >> select(_.feed_key, _.trip_id, _.stop_id, _.stop_sequence)
        trip_with_pair = trip_with_pair >> inner_join(_, warehouse_data['stops'] >> select(_.feed_key, _.stop_id, _.geometry),
                                                      on = ['feed_key', 'stop_id'])
        trip_with_pair = trip_with_pair >> inner_join(_,
                                                      warehouse_data['trips'] >> select(_.feed_key, _.name, _.trip_id, _.shape_id,
                                                                            _.route_short_name, _.route_long_name, _.route_id),
                                                      on = ['feed_key', 'trip_id'])
        paired_shape = warehouse_data['shapes'] >> filter(_.feed_key == trip_with_pair.feed_key.iloc[0], _.shape_id == trip_with_pair.shape_id.iloc[0])
            
        if not trip_with_pair.stop_id.is_unique:
            if verbose:
                print('warning, trip has duplicate stops at a single stop')
            trip_with_pair = trip_with_pair >> distinct(_.stop_id, _keep_all=True)
        stop0 =  (trip_with_pair >> filter(_.stop_sequence == _.stop_sequence.min())).geometry.iloc[0]
        stop1 =  (trip_with_pair >> filter(_.stop_sequence == _.stop_sequence.max())).geometry.iloc[0]
        # display(trip_with_pair)
        # print(stop0, stop1)
        if paired_shape.empty:
            if verbose:
                print('warning, trip has no shape')
            trip_with_pair = trip_with_pair >> distinct(_.stop_id, _keep_all=True)
            paired_segment = LineString([stop0, stop1])
        # stop0_proj = shape_geom.project(stop0)
        # stop1_proj = shape_geom.project(stop1)
        else:
            shape_geom = paired_shape.geometry.iloc[0]
            stops_proj = [shape_geom.project(stop0), shape_geom.project(stop1)] #  be resillient to looping
            paired_segment = substring(shape_geom, min(stops_proj), max(stops_proj))
        
        trip_with_pair['segment_geom'] = paired_segment
        trip_with_pair.set_geometry('segment_geom')
        trip_with_pair = trip_with_pair >> rename(stop_geom = _.geometry)
        # display(stop_pair)
        trip_with_pair = trip_with_pair >> distinct(_.shape_id, _keep_all=True)
        trip_with_pair['stop_pair'] = [stop_pair]
        trip_with_pair['trip_group_id'] = row.trip_group_id
        trip_with_pair['optimal_pct'] = row.nIterations / row.total_iterations #  more human-friendly name
        # percentige of time in analysis window that this route was optimal
        trip_with_pair['total_time'] = row.totalTime
        trip_with_pair['xfer_count'] = len(stop_pairs) - 1
        trip_with_seg = gpd.GeoDataFrame(trip_with_pair, geometry='segment_geom', crs=geography_utils.CA_NAD83Albers_m)
        row_shape_segments += [trip_with_seg]
        
        spatial_routes = pd.concat(row_shape_segments)
        spatial_routes['route_name'] = spatial_routes.route_short_name.fillna(spatial_routes.route_long_name)
        
    return spatial_routes

In [34]:
for _ix, row in df2.iterrows():
    test_ssr2 = shape_segments_from_row(row, warehouse_data, True)
    break    

In [40]:
test_ssr2.groupby('trip_id')[['stop_id']].count().reset_index().rename(columns={'stop_id':'n'}).query('n > 1')

,trip_id,n
0,10209000510529-JUNE23,2
1,10209000510622-JUNE23,2
2,10209000510722-JUNE23,2
3,10209000510822-JUNE23,2
4,10209000510922-JUNE23,2
5,10209000511022-JUNE23,2
6,10209000511122-JUNE23,2
7,10209000511222-JUNE23,2
8,10209000511321-JUNE23,2
9,10209000511421-JUNE23,2


In [36]:
test_ssr2 >> count(_.trip_id) >> filter(_.n > 1)

,trip_id,n
0,10209000510529-JUNE23,2
1,10209000510622-JUNE23,2
2,10209000510722-JUNE23,2
3,10209000510822-JUNE23,2
4,10209000510922-JUNE23,2
5,10209000511022-JUNE23,2
6,10209000511122-JUNE23,2
7,10209000511222-JUNE23,2
8,10209000511321-JUNE23,2
9,10209000511421-JUNE23,2


In [35]:
test_ssr2

,key,_gtfs_key,base64_url,feed_key,trip_id,stop_id,stop_sequence,arrival_time,departure_time,arrival_time_interval,...,start_pickup_drop_off_window_interval,end_pickup_drop_off_window_interval,start_pickup_drop_off_window_sec,end_pickup_drop_off_window_sec,mean_duration_factor,mean_duration_offset,safe_duration_factor,safe_duration_offset,pickup_booking_rule_id,drop_off_booking_rule_id
2592,9f2fbf33a257366c010d972d0694ac12,e77183eeb8cd380d02b6fd4cd468be90,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000511321-JUNE23,141078,32,13:51:00,13:51:00,"relativedelta(hours=+13, minutes=+51)",...,None,None,None,None,None,None,None,None,None,None
2593,4db0b8f029f6ccf13b53a3bb63571216,9bdca4081b284051d5f0c7b700b11696,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000511923-JUNE23,141078,32,19:51:00,19:51:00,"relativedelta(hours=+19, minutes=+51)",...,None,None,None,None,None,None,None,None,None,None
2594,a3938eeb540ef11bae77a9b7c6b82375,f0dbe651699cba3d333a04efe2a976de,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000511122-JUNE23,141078,32,11:51:00,11:51:00,"relativedelta(hours=+11, minutes=+51)",...,None,None,None,None,None,None,None,None,None,None
2595,0f30730a9a02a7592c156e58607dab21,16647708a0c49427a783d3634ad0c52e,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000512027-JUNE23,141078,32,20:55:00,20:55:00,"relativedelta(hours=+20, minutes=+55)",...,None,None,None,None,None,None,None,None,None,None
2596,3deaf85f611f10b58dffeaf6b52b1943,dfcc4c00a293357a3240974e6cb81f75,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000511521-JUNE23,141078,32,15:51:00,15:51:00,"relativedelta(hours=+15, minutes=+51)",...,None,None,None,None,None,None,None,None,None,None
2597,cd278f160617c5881f5b323978c19464,d27ffe9acff2617b88e741cb0cc01170,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000511621-JUNE23,141078,32,16:51:00,16:51:00,"relativedelta(hours=+16, minutes=+51)",...,None,None,None,None,None,None,None,None,None,None
2639,e01e4273fa3ee11e1466694fb17e1f59,43c11c7cc3a9abd0d1e26e1aa1d697fa,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000511122-JUNE23,2489,44,12:05:00,12:05:00,"relativedelta(hours=+12, minutes=+5)",...,None,None,None,None,None,None,None,None,None,None
2644,952404456ea28c6d463585b9b84c0f79,ef6aa2b1cd28d553274fefb1e1ff630c,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000511521-JUNE23,2489,44,16:07:00,16:07:00,"relativedelta(hours=+16, minutes=+7)",...,None,None,None,None,None,None,None,None,None,None
2645,9bd05318f99e7ba7533d005b9918a5d3,8cfde55a1b2a02ffb71db30766a83558,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000511923-JUNE23,2489,44,20:05:00,20:05:00,"relativedelta(hours=+20, minutes=+5)",...,None,None,None,None,None,None,None,None,None,None
2647,d59900d347dd97adaae1f6bb59233693,f44a2214a0c32d3b3df397fe57ffd531,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,edb0fd72b98a7f739407ceca3fae9034,10209000512027-JUNE23,2489,44,21:08:00,21:08:00,"relativedelta(hours=+21, minutes=+8)",...,None,None,None,None,None,None,None,None,None,None
